# Fourier transform

The Fourier transform of a periodic function $p(t)$ with period $T_0$ is computed as

$$p(t) = a_0 + \sum_{j=1}^{\infty} a_j\cos{j\omega_0t} + \sum_{j=1}^{\infty} b_j\sin{j\omega_0t}$$

where

$$\omega_0 = \frac{2\pi}{T_0}$$ is the angular frequency of the periodic function

$$a_0 = \frac{1}{T_0} \int_0^{T_0} p(t)dt$$

$$a_j = \frac{2}{T_0} \int_0^{T_0} p(t)\cos{j\omega_0t}dt$$

$$b_j = \frac{2}{T_0} \int_0^{T_0} p(t)\sin{j\omega_0t}dt$$

Alternatively, the Fourier transform can be expressed in terms of complex exponential functions as

$$p(t) = \sum_{j=-\infty}^{\infty} P(j\omega_0)e^{ij\omega_0t}$$

where

$$i = \sqrt{-1}$$

$$P(j\omega_0) = \frac{1}{T_0} \int_0^{T_0} p(t)e^{-ij\omega_0t}dt$$

Although each of these expressions involve sums over infinite harmonic components, a reasonable approximation of the function $p(t)$ can often be achieved using just the first few harmonic components.

This notebook computes and plots the Fourier transform of a square and sawtooth wave function.

In [1]:
% matplotlib inline

# Import required modules
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
import ipywidgets as wid

# Define plot attributes
plt.rcParams['figure.figsize'] = (16, 5)
plt.rcParams['font.size'] = 16
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['xtick.labelsize'] = 'small'
plt.rcParams['ytick.labelsize'] = 'small'

In [2]:
"""
Compute the square wave function

p0 : Amplitude of the function
T  : Period of the function
t  : Domain at which to evaluate the function
"""
def square_wave(p0, T, t):
    return p0*np.sign(np.sin(2.0*np.pi*t/T))

"""
Compute the sawtooth wave function

p0 : Amplitude of the function
T  : Period of the function
t  : Domain at which to evaluate the function
"""
def sawtooth_wave(p0, T, t):
        return 2.0*p0*(t/T - np.floor(t/T + 1.0/2.0))

# Define the function whose Fourier transform is to be computed
p0 = 1.0
T = 1.0
num_pts = 512
t = np.linspace(0.0, T, num_pts)
#p = square_wave(p0, T, t)
p = sawtooth_wave(p0, T, t)

In [3]:
"""
Compute and plot the Fourier transform of a given function

num_harm : Number of harmonics to include (must be lesser than half the number of elements in t)
p0       : Amplitude of the function
T        : Period of the function
p_name   : Name of the function whose Fourier transform is to be computed and plotted
"""
def compute_plot_fourier_transform(p0, T, p_name, num_harm, num_rep):
    
    # Compute the function over one period
    num_pts = 512
    t = np.linspace(0.0, T, num_pts)
    func_map = {'Square wave':square_wave, 'Sawtooth wave':sawtooth_wave}
    p = func_map[p_name](p0, T, t)
    
    # Compute the plotting domain and plot the function itself
    tplot = []
    pplot = []
    for i in range(num_rep):
        tplot += [t + i*t[-1]]
        pplot += [p]
    tplot = np.hstack(tplot)
    pplot = np.hstack(pplot)
    
    plt.plot(tplot, pplot, lw=2.0, color='k')

    # Compute the fourier transform
    num_pts = len(t)
    P = scipy.fftpack.fft(p)/num_pts
    
    # Plot the contant component
    fourier = np.real(P[0])*np.ones(num_rep*num_pts)
    plt.plot(tplot, fourier, lw=1.0, color='0.7')
    
    # Plot the first n harmonic components
    a = 2.0*np.real(P)[1:num_harm + 1]
    b = -2.0*np.imag(P)[1:num_harm + 1]
    omega = 2.0*np.pi/T
    for i in range(1, num_harm + 1):
        harmonic = a[i - 1]*np.cos(i*omega*tplot) + b[i - 1]*np.sin(i*omega*tplot)
        plt.plot(tplot, harmonic, lw=1.0, color='0.7')
        fourier += harmonic
    
    # Plot the fourier transform of the function
    plt.plot(tplot, fourier, lw=2.0, color='#BF1C1C')
    
    # Finalise plot
    plt.grid(False)
    plt.xlim(tplot[0], tplot[-1])
    plt.xlabel('$t/T$')
    plt.ylabel('$p(t)/p_0$')
    
    plt.tight_layout(pad=0.5)
    plt.show()

# Create the sliders
wid.interact(compute_plot_fourier_transform, \
        p0=wid.fixed(1.0), \
        T=wid.fixed(1.0), \
        p_name=wid.ToggleButtons(value='Sawtooth wave', options=('Square wave', 'Sawtooth wave'), description='Function type'), \
        num_harm=wid.IntSlider(value=10, min=0, max=20, step=1, description='No. of harmonics', continuous_update=False), \
        num_rep=wid.IntSlider(value=4, min=1, max=6, step=1, description='No. of periods', continuous_update=False))

interactive(children=(ToggleButtons(description='Function type', index=1, options=('Square wave', 'Sawtooth wa…

<function __main__.compute_plot_fourier_transform>